<a href="https://colab.research.google.com/github/Ansil-bayan/MachineTranslation_en_to_ml/blob/main/MachineTranslation_en_to_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q transformers torch peft datasets sacrebleu evaluate

In [3]:
from transformers import MBartForConditionalGeneration, MBart50Tokenizer, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments

In [4]:
from datasets import load_dataset
import evaluate

In [5]:
import numpy as np
import torch, os
os.environ["WANDB_DISABLED"] = "true"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
raw_dataset = load_dataset("ai4bharat/samanantar","ml",split="train[:1000]", trust_remote_code=True)

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'ai4bharat/samanantar' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
ERROR:datasets.load:`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'ai4bharat/samanantar' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able t

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/196M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/196M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/196M [00:00<?, ?B/s]

train-00003-of-00004.parquet:   0%|          | 0.00/196M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5924426 [00:00<?, ? examples/s]

In [7]:
dataset = raw_dataset.take(1000)
dataset

Dataset({
    features: ['idx', 'src', 'tgt'],
    num_rows: 1000
})

In [8]:
tokenized_dataset = dataset.train_test_split(test_size=0.1)

In [9]:
train_dataset =tokenized_dataset["train"]
eval_dataset = tokenized_dataset["test"]
print(train_dataset[0])
print(eval_dataset[0])

{'idx': 45, 'src': 'But India won.', 'tgt': 'പക്ഷേ, ഇന്ത്യ ജയിച്ചു.'}
{'idx': 621, 'src': 'Thats the strength of this movie.', 'tgt': 'ഇതുതന്നെയാണ് ചിത്രത്തിന്റെ കരുത്തും.'}


In [10]:
model_name = "facebook/mbart-large-50-many-to-many-mmt"
model = MBartForConditionalGeneration.from_pretrained(model_name)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [11]:
tokenizer = MBart50Tokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [12]:
def preprocess_function(a):           #to convert train dataset and eval dataset into token numbers
  inputs = [i for i in a['src']]
  targets = [i for i in a['tgt']]

  model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

  labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length").input_ids
  labels = [[-100 if token == tokenizer.pad_token_id else token for token in label] for label in labels]

  model_inputs["labels"] = labels
  return model_inputs

tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [13]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy= "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    fp16=True,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=10,
    generation_max_length=128,
    report_to="none"

)

In [14]:
metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
  preds,labels = eval_preds

  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

  labels = np.where(labels!=-100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  result = metric.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])

  return result

In [15]:
trainer = Seq2SeqTrainer(
    model=model,
    args= training_args,
    train_dataset= tokenized_train_dataset,
    eval_dataset= tokenized_eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-3229105591.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [16]:
save_directory = "/kaggle/new_model/saved_model"

os.makedirs(save_directory, exist_ok=True)

In [17]:
train_result=trainer.train()

Epoch,Training Loss,Validation Loss,Score,Counts,Totals,Precisions,Bp,Sys Len,Ref Len
1,3.381000,3.026621,2.021471,"[218, 37, 8, 0]","[820, 720, 621, 527]","[26.585365853658537, 5.138888888888889, 1.288244766505636, 0.09487666034155598]",1.000000,820,819
2,2.682600,2.970365,1.963093,"[211, 41, 9, 0]","[871, 771, 673, 580]","[24.225028702640643, 5.317769130998703, 1.337295690936107, 0.08620689655172414]",1.000000,871,819
3,2.301200,2.967742,2.756126,"[222, 47, 13, 1]","[852, 752, 654, 561]","[26.056338028169016, 6.25, 1.9877675840978593, 0.17825311942959002]",1.000000,852,819


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3854: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


In [18]:
trainer.save_model(save_directory)

In [19]:
train_result.metrics

{'train_runtime': 575.3872,
 'train_samples_per_second': 4.692,
 'train_steps_per_second': 0.297,
 'total_flos': 731406296678400.0,
 'train_loss': 2.905190460863169,
 'epoch': 3.0}

In [21]:
save_directory = './finetuned-MBart50-en-ml'

# Save model and tokenizer
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"Model saved to {save_directory}")

Model saved to ./finetuned-MBart50-en-ml


In [22]:
# @title Translate {"display-mode":"form"}

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Load your model and tokenizer
model_name =save_directory
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Example English sentence to translate
English = "Thats the film's script" # @param {"type":"string"}

# Tokenize the input sentence
inputs = tokenizer(English, return_tensors="pt").to(device)

# Generate translation
with torch.no_grad():
    outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)

# Decode the generated tokens
translated_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"English: {English}")
print(f"Mal Translation: {translated_sentence}")

English: Thats the film's script
Mal Translation: അതാണ് ചിത്രത്തിന് റെ തിരക്കഥ.
